In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [1]:
!pip install  langchain
!pip install pdfplumber
!pip install accelerate
!pip install CTransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 56.8 MB/s eta 0:00:00


In [2]:
from langchain.llms import LlamaCpp
from langchain import LLMChain
from langchain.prompts import PromptTemplate
import json
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain.llms import CTransformers
import traceback
import pdfplumber
from accelerate import Accelerator
import random

In [3]:
template = """
Text: {text}
You are an expert question maker. Given the above text, it is your job to create a quiz of {number} questions for grade {grade} students in {tone} tone, with question type: {question_type}.
Make sure that questions are not repeated and check all the questions to be conforming to the text as well.
Make sure to format your response like the RESPONSE_JSON below and use it as a guide. Ensure to make the {number} questions and provide answers for every question.
### RESPONSE_JSON
{response_json}
"""
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "grade", "tone","question_type", "response_json"],
    template=template,
)

In [4]:
a=Accelerator()

In [5]:
config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 4000, 'temperature':0, 'gpu_layers':50}

In [ ]:
llm = CTransformers(model=r"/content/drive/MyDrive/Projectl/Model/llama-2-7b-chat.ggmlv3.q4_1.bin",model_type="llama", gpu_layers=50, config=config)

In [ ]:
llm, config = a.prepare(llm, config)

In [ ]:
llm_chain = LLMChain(prompt=quiz_generation_prompt, llm=llm, verbose=True,output_key="quiz")

In [ ]:
template = """You are an expert english grammarian and writer. Given a multiple choice quiz for {grade} grade students.\
You need to evaluate complexity of the questions and give a complete analysis of the quiz if the students
will be able to understand the questions and answer them. Only use at max 50 words for complexity analysis.
If quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed and change the tone such that it perfectly fits the students abilities.
Quiz MCQs:
{quiz}
Critique from an expert english writer of the above quiz:"""


quiz_evaluation_prompt = PromptTemplate(
    input_variables=["grade", "quiz"], template=template
)
review_chain = LLMChain(
    llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True
)

In [ ]:
generate_evaluate_chain = SequentialChain(
    chains=[llm_chain, review_chain],
    input_variables=["text", "number", "grade", "tone","question_type", "response_json"],
    # Here we return multiple variables
    output_variables=["quiz", "review"],
    verbose=True,
)

In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file_path):
    text = ""
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Define the path to your PDF file
pdf_file_path = "/content/drive/MyDrive/Projectl/Big Mac Index.pdf"

question_types = ["multiple choice", "true or false", "Generate a question for one word answer"]


In [ ]:
extracted_text=extract_text_from_pdf(pdf_file_path)
num_questions=10
grade=5
tone="formal"
question_type= "multiple choice"


In [ ]:
RESPONSE_JSON = {}
for i in range(1, num_questions + 1):
    question_data = {}

    if question_type == "multiple choice":
        question_data = {
            "no": str(i),
            "mcq": "multiple choice question",
            "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here"
            },
            "correct": "correct answer"
        }
    elif question_type == "true or false":
        question_data = {
            "no": str(i),
            "true_or_false": "true or false question",
            "correct": "correct answer"  # Specify the correct answer
        }
    elif question_type == "Question for single word answer":
        question_data = {
            "no": str(i),
            "one_word_answer": "Generate a question for one-word answer",
            "correct_answer": "correct answer"  # Specify the correct answer
        }

    RESPONSE_JSON[str(i)] = question_data

In [ ]:
input_variables = {
    "text": extracted_text,
    "number": num_questions,
    "grade": grade,
    "tone": tone,
    "question_type": question_type,
    "response_json": json.dumps(RESPONSE_JSON),
}

In [ ]:
output_variables = generate_evaluate_chain(input_variables)
